# Setup

In [1]:
import pandas as pd
import numpy as np
import re
import os

import warnings
warnings.filterwarnings('ignore')


## Import datasets

Leggo e printo le prime righe del dataset per controllare sia a posto

In [ ]:
N = 30
with open("./data/CSV/BIC_VOCABOLARIO_202209082053.csv") as myfile:
    head = [next(myfile) for x in range(N)]
print(head)

### VOCABOLARIO

In [2]:
# vocabolario = pd.read_csv("BIC_VOCABOLARIO_prime4.csv", sep="*") # test
vocabolario = pd.read_csv("./data/CSV/BIC_VOCABOLARIO_202209082053.csv", sep="\t", quotechar = "%")
vocabolario.head()

,ID_VOCABOLO,ID_OPERA,VOCABOLO,SEPARATORE
0,243294066,2,Canto,S1P1P
1,243294067,2,Le,S
2,243294068,2,donne,",S"
3,243294069,2,i,S
4,243294070,2,cavallier,",S"


In [ ]:
# check
# vocabolario[vocabolario['ID_VOCABOLO'] == 121442978]

### OPERE

In [3]:
opere = pd.read_csv("./data/CSV/BIC_OPERE_202209090122.csv", sep="\t", quotechar = "%", parse_dates=True)
opere['ANNO_FINE_STESURA'] = opere['FINE_STESURA'].str[:4].astype('Int64')
opere['ANNO_PUBBLICAZIONE'] = opere['PUBBLICAZIONE'].str[:4].astype('Int64')
print(opere.dtypes)
opere.head()


ID_OPERA               int64
ID_AUTORE              int64
ID_GENERE              int64
TITOLO                object
FINE_STESURA          object
PUBBLICAZIONE         object
ANNO_FINE_STESURA      Int64
ANNO_PUBBLICAZIONE     Int64
dtype: object


,ID_OPERA,ID_AUTORE,ID_GENERE,TITOLO,FINE_STESURA,PUBBLICAZIONE,ANNO_FINE_STESURA,ANNO_PUBBLICAZIONE
0,2,3,2130,Orlando furioso,NaN,1532-01-31,<NA>,1532
1,3,4,2300,Novelle,NaN,1554-01-01,<NA>,1554
2,4,5,2300,Decameron,1353-12-31,NaN,1353,<NA>
3,5,6,2130,Orlando innamorato,NaN,1494-12-31,<NA>,1494
4,6,7,2210,Margherita Pusterla,NaN,1838-01-01,<NA>,1838


### GENERI

In [4]:
generi = pd.read_csv("./data/CSV/BIC_GENERI_202209090122.csv", sep="\t", quotechar = "%", parse_dates=True)
print(generi.dtypes)
generi.head()

ID_GENERE     int64
ID_TESTO      int64
GENERE       object
dtype: object


,ID_GENERE,ID_TESTO,GENERE
0,0,0,[inclassificato]
1,4050,2501,carteggio
2,1001,1001,[lirica grave]
3,1501,1001,[lirica leggera]
4,2110,2001,poema eroico


In [14]:
generi['ID_TESTO'].unique()

array([   0, 2501, 1001, 2001, 4501, 4001, 5501, 5001, 3001], dtype=int64)

### AUTORI

In [5]:
autori = pd.read_csv("./data/CSV/BIC_AUTORI_202209090121.csv", sep="\t", quotechar = "%", parse_dates=True)
autori['ANNO_DATA_NASCITA'] = autori['DATA_NASCITA'].str[:4].astype('Int64')
autori['ANNO_DATA_MORTE'] = autori['DATA_MORTE'].str[:4].astype('Int64')
print(autori.dtypes)
autori.head()

ID_AUTORE             int64
NOME                 object
COGNOME              object
DATA_NASCITA         object
DATA_MORTE           object
ANNO_DATA_NASCITA     Int64
ANNO_DATA_MORTE       Int64
dtype: object


,ID_AUTORE,NOME,COGNOME,DATA_NASCITA,DATA_MORTE,ANNO_DATA_NASCITA,ANNO_DATA_MORTE
0,2,Dante,Alighieri,1265-05-22,1321-09-14,1265,1321
1,3,Ludovico,Ariosto,1474-09-08,1533-07-06,1474,1533
2,4,Matteo,Bandello,1485-01-01,1561-12-31,1485,1561
3,5,Giovanni,Boccaccio,1313-06-16,1375-12-21,1313,1375
4,6,Matteo Maria,Boiardo,1441-01-01,1494-12-19,1441,1494


### TESTI

In [6]:
testi = pd.read_csv("./data/CSV/BIC_TESTI_202209120226.csv", sep="\t", quotechar = "%", parse_dates=True)
#testi['ANNO_DATA_NASCITA'] = testi['DATA_NASCITA'].str[:4].astype('Int64')
#testi['ANNO_DATA_MORTE'] = testi['DATA_MORTE'].str[:4].astype('Int64')
print(testi.dtypes)
testi.head()

ID_TESTO            int64
CLASSIFICAZIONE    object
dtype: object


,ID_TESTO,CLASSIFICAZIONE
0,0,testo inclassificabile
1,1001,testo espressivo
2,2001,testo riferitivo con patto finzionale
3,2501,testo riferitivo senza patto finzionale
4,3001,testo dichiarativo


# Merging

Per prima cosa unisco in un'unica tabella le meta-information rilevanti riguardanti ogni opera

In [7]:
opere_autori = pd.merge(opere, autori, on='ID_AUTORE', how='left')
opere_autori['AUTORE'] = opere_autori['NOME'] + ' ' + opere_autori['COGNOME']
opere_autori_generi = pd.merge(opere_autori, generi, on='ID_GENERE', how='left')
opere_autori_generi_testi = pd.merge(opere_autori_generi, testi, on='ID_TESTO', how='left')
opere_autori_generi_testi.head()

,ID_OPERA,ID_AUTORE,ID_GENERE,TITOLO,FINE_STESURA,PUBBLICAZIONE,ANNO_FINE_STESURA,ANNO_PUBBLICAZIONE,NOME,COGNOME,DATA_NASCITA,DATA_MORTE,ANNO_DATA_NASCITA,ANNO_DATA_MORTE,AUTORE,ID_TESTO,GENERE,CLASSIFICAZIONE
0,2,3,2130,Orlando furioso,NaN,1532-01-31,<NA>,1532,Ludovico,Ariosto,1474-09-08,1533-07-06,1474,1533,Ludovico Ariosto,2001,poema cavalleresco,testo riferitivo con patto finzionale
1,3,4,2300,Novelle,NaN,1554-01-01,<NA>,1554,Matteo,Bandello,1485-01-01,1561-12-31,1485,1561,Matteo Bandello,2001,novella o racconto,testo riferitivo con patto finzionale
2,4,5,2300,Decameron,1353-12-31,NaN,1353,<NA>,Giovanni,Boccaccio,1313-06-16,1375-12-21,1313,1375,Giovanni Boccaccio,2001,novella o racconto,testo riferitivo con patto finzionale
3,5,6,2130,Orlando innamorato,NaN,1494-12-31,<NA>,1494,Matteo Maria,Boiardo,1441-01-01,1494-12-19,1441,1494,Matteo Maria Boiardo,2001,poema cavalleresco,testo riferitivo con patto finzionale
4,6,7,2210,Margherita Pusterla,NaN,1838-01-01,<NA>,1838,Cesare,Cantù,1804-12-05,1895-03-15,1804,1895,Cesare Cantù,2001,romanzo,testo riferitivo con patto finzionale


### Test

In [57]:
# test = list(map(str, opere_autori_generi_testi[opere_autori_generi_testi['ID_OPERA'] == 2].values.tolist()[0]))
# print(",".join(test) + "\n" + "ciao")

[2, 3, 2130, 'Orlando furioso', nan, '1532-01-31', <NA>, 1532, 'Ludovico', 'Ariosto', '1474-09-08', '1533-07-06', 1474, 1533, 'Ludovico Ariosto', 2001, 'poema cavalleresco', 'testo riferitivo con patto finzionale']
ciao


In [49]:
# meta_columns = ['TITOLO', 'AUTORE','GENERE','CLASSIFICAZIONE','ANNO_FINE_STESURA','ANNO_PUBBLICAZIONE']
# ",".join(meta_columns)

'TITOLO,AUTORE,GENERE,CLASSIFICAZIONE,ANNO_FINE_STESURA,ANNO_PUBBLICAZIONE'

In [71]:
# opere_autori_generi_testi[opere_autori_generi_testi['ID_OPERA'] == 35]
# print(opere_autori_generi_testi['ID_OPERA'].unique())

[   2    3    4 ... 9429 9432 9440]


# Books Generator

In [4]:
# Faccio una merge in modo da avere il titolo dell'opera nella funzione
#vocabolario = pd.merge(vocabolario, opere[['ID_OPERA', 'TITOLO']], left_on='ID_OPERA', right_on='ID_OPERA', how='left', copy=False)

In [101]:
def rebuild_text(vocabolario_,meta_data, path): # passare dataframe completo
    id_opere = vocabolario_['ID_OPERA'].unique().tolist() # estraggo id_opere unici

    for i in id_opere:
        df_vocabolario = vocabolario_[vocabolario_['ID_OPERA'] == i]
        
        # Sostituisco eventuali NaN con la sigla MW (Missing Word) o MS (Missing Separator)
        df_vocabolario.VOCABOLO = df_vocabolario.VOCABOLO.fillna('MW')
        df_vocabolario.SEPARATORE = df_vocabolario.SEPARATORE.fillna('MS')
        
        # Traduco le lettere nei separatori
        df_vocabolario['SEPARATORE'] = df_vocabolario['SEPARATORE'].str.replace('S',' ', regex=False)
        df_vocabolario['SEPARATORE'] = df_vocabolario['SEPARATORE'].str.replace('P','\n', regex=False) # uso \n per andare a capo quando scrivo il .txt
        
        # Creo la lista con VOCABOLO + SEPARATORE
        word_list = df_vocabolario['VOCABOLO']
        sep_list = df_vocabolario['SEPARATORE']
        result = [None]*(len(word_list)+len(sep_list))
        result[::2] = word_list
        result[1::2] = sep_list
        
        # Creo il dataframe necessario, con manipolazione
        df_2 = pd.DataFrame(result)
        df_2.rename(columns={0: "PAROLA"}, inplace=True)
        df_2['PARAGRAFO'] = df_2["PAROLA"].str.contains('\n', regex=False).cumsum()
        df_2['PARAGRAFO'] = df_2['PARAGRAFO'].where(~df_2['PAROLA'].str.contains('\n', regex=False), df_2['PARAGRAFO']-1)
                        # Riduco di 1 il valore di paragrafo quando ho l'escape character \n per andare a capo nel punto giusto
        
        # Creo il dataframe per costruire le frasi, con manipolazione
        groups = df_2.groupby("PARAGRAFO")
        sentences = groups.apply(lambda sdf: "".join(sdf.PAROLA))
        df_3 = pd.DataFrame(sentences)
        df_3.reset_index(inplace=True)
        df_3.drop('PARAGRAFO', axis=1, inplace=True)
        
        # Meta data
        meta_columns = ['TITOLO', 'AUTORE','GENERE','CLASSIFICAZIONE','ANNO_FINE_STESURA','ANNO_PUBBLICAZIONE']
        df_meta_data = meta_data[meta_data['ID_OPERA'] == i][meta_columns]
        # print("OPERA " + str(i))
        # print(df_meta_data)
        if len(df_meta_data) != 0:
            text_title = re.sub(r'[\\/*?:"<>|/]',"",df_meta_data['TITOLO'].unique().tolist()[0]).strip() # Rendo il file salvabile (elimino forbidden char)
            text_title = text_title[:175]
            meta_info = list(map(str, df_meta_data.values.tolist()[0]))
        else:
            text_title = re.sub(r'[\\/*?:"<>|]',"",'0_UnknownOperaID' + str(i)).strip()
        # Creo il .txt
        try:
            with open(path + text_title + '.txt', "r", encoding='utf-8') as file:
                # print('pass')
                pass  
        except FileNotFoundError:
            with open(path + text_title + '.txt', "w", encoding='utf-8') as file:
                # Meta information
                if len(df_meta_data) != 0:
                    file.write(",".join(meta_columns) + "\n")
                    file.write(",".join(meta_info) + "\n\n")
                else:
                    file.write("\n\n\n")
                    print('OPERA ' + str(i) + ' non ha un match in df_meta_data')
                # Testo
                for index, row in df_3.iterrows():
                    file.write(row[0])
        finally:
            pass

In [6]:
# Mi sposto nella cartella "Books" per salvare i libri
# import os
# os.chdir('Books')

In [102]:
rebuild_text(vocabolario, opere_autori_generi_testi, "./data/books/")

OPERA 6240 non ha un match in df_meta_data
OPERA 8399 non ha un match in df_meta_data


## Correzione errori

### Check conteggio

Sono stati generati 3001 libri, di cui 8 senza titolo (senza match in BIC_OPERE), rispetto ai 3118 iniziali

In [103]:
# Conteggio libri unici / verifico che matchino con i libri printati
vocabolario['ID_OPERA'].nunique()

3118

In [107]:
len(vocabolario['ID_OPERA'].unique().tolist())

3118

In [109]:
# Conteggio libri unici nel dataset delle opere
# La differenza è di 8 libri, esattamente i 8 sconosciuti
opere_autori_generi_testi['ID_OPERA'].nunique()

3111

In [108]:
# Controllo che non ci siano delle opere con medesimo nome (Questo potrebbe aver
# portato a ottenere solo 3001 libri finali)
opere_autori_generi_testi['TITOLO'].nunique()

2997

In [23]:

len(opere_autori_generi_testi[opere_autori_generi_testi['GENERE'] == 'carteggio'])

12

# Aggiunta libri dal 1922

Aggiunta line-break dopo ogni punto nei file

In [86]:
# test

with open(".\data\\books_after_1922\\test\CESARE PAVESE. - Eriberto.txt", "r") as file:
    filedata = file.read()
# prima tolgo interruzioni non relative a punti
filedata = filedata.replace('\n', '')
# poi aggiunto line-break dopo ogni punto
filedata = filedata.replace('.', '.\n')
with open(".\data\\books_after_1922\\test\CESARE PAVESE. - Eriberto_1.txt", 'w') as file:
  file.write(filedata)

Trattamento dei libri:
- Elimino tutti i line break (anche quelli scorretti) -> Elimino solo i single line break e mantengo quelli > 1 (dei titoli o altro)
- Aggiungo un line-break dopo ogni punto

In [93]:
path_libri_non_trattati = '.\data\\books_after_1922\\libri_non_trattati'
path_libri_trattati = '.\data\\books_after_1922\\libri_trattati'

for filename in os.listdir(path_libri_non_trattati):
    filedata = None # azzero
    with open(f"{path_libri_non_trattati}\{filename}", "r", encoding="utf8") as file:
        filedata = file.read()
    # prima tolgo interruzioni non relative a punti
    # filedata = filedata.replace('\n', '')
    filedata = re.sub(r"(?<!\n)\n(?!\n)", " ", filedata)
    # poi aggiunto line-break dopo ogni punto
    filedata = filedata.replace('.', '.\n')
    with open(f"{path_libri_trattati}\{filename}", 'w', encoding="utf8") as file:
        file.write(filedata)

## Creazione tabella libri aggiuntivi

Importo i nomi di tutti i libri per poi:
- Generare un dataframe a una colonna
- Splittare la data in una prima colonna
- Splttare Titolo e Autore
- Mantenendo nome file

In [8]:
path_libri_trattati = '.\data\\books_after_1922\\libri_trattati'
lista_libri_trattati = os.listdir(f"{path_libri_trattati}\\tutti")
opere_aggiuntive = pd.DataFrame(lista_libri_trattati, columns = ['Filename'])
opere_aggiuntive.head()

,Filename
0,"1923 La coscienza di Zeno, Italo Svevo.txt"
1,"1926 Uno, nessuno e centomila, Luigi Pirandell..."
2,"1930 Gente in Aspromonte, Corrado Alvaro.txt"
3,"1933 Fontamara, Ignazio Silone.txt"
4,"1934 Sorelle Materassi, Aldo Palazzeschi.txt"


In [9]:
opere_aggiuntive[['ANNO_RIFERIMENTO','altro']] = opere_aggiuntive['Filename'].str.split(' ', 1, expand=True)
opere_aggiuntive[['TITOLO','AUTORE']] = opere_aggiuntive['altro'].str.split(', ', 1, expand=True)
opere_aggiuntive['AUTORE'] = opere_aggiuntive['AUTORE'].str.replace(".txt","")
opere_aggiuntive.drop('altro',1,inplace=True)
opere_aggiuntive = opere_aggiuntive[['ANNO_RIFERIMENTO','TITOLO','AUTORE','Filename']]
opere_aggiuntive['ANNO_RIFERIMENTO'] = opere_aggiuntive['ANNO_RIFERIMENTO'].astype('Int64')
opere_aggiuntive.head(10)

,ANNO_RIFERIMENTO,TITOLO,AUTORE,Filename
0,1923,La coscienza di Zeno,Italo Svevo,"1923 La coscienza di Zeno, Italo Svevo.txt"
1,1926,Uno,"nessuno e centomila, Luigi Pirandello","1926 Uno, nessuno e centomila, Luigi Pirandell..."
2,1930,Gente in Aspromonte,Corrado Alvaro,"1930 Gente in Aspromonte, Corrado Alvaro.txt"
3,1933,Fontamara,Ignazio Silone,"1933 Fontamara, Ignazio Silone.txt"
4,1934,Sorelle Materassi,Aldo Palazzeschi,"1934 Sorelle Materassi, Aldo Palazzeschi.txt"
5,1937,Novelle per un anno,Luigi Pirandello,"1937 Novelle per un anno, Luigi Pirandello.txt"
6,1938,Il mulino del Po,Riccardo Bacchelli,"1938 Il mulino del Po, Riccardo Bacchelli.txt"
7,1939,La pietra lunare,Tommaso Landolfi,"1939 La pietra lunare, Tommaso Landolfi.txt"
8,1940,Il deserto dei tartari,Dino Buzzati,"1940 Il deserto dei tartari, Dino Buzzati.txt"
9,1944,Kaputt,Curzio Malaparte,"1944 Kaputt, Curzio Malaparte.txt"


# Corpus

## Divisione per Periodo storico

### Generazione colonne relative al periodo storico

Aggiungo un anno di riferimento danno preferenzialità a ANNO_PUBBLICAZIONE

In [7]:
conditions = [
    (opere_autori_generi_testi['ANNO_PUBBLICAZIONE'].notnull()),
    (~opere_autori_generi_testi['ANNO_PUBBLICAZIONE'].notnull() & opere_autori_generi_testi['ANNO_FINE_STESURA'].notnull()),
    (~opere_autori_generi_testi['ANNO_PUBBLICAZIONE'].notnull() & ~opere_autori_generi_testi['ANNO_FINE_STESURA'].notnull()),
    ]
values = [opere_autori_generi_testi['ANNO_PUBBLICAZIONE'], opere_autori_generi_testi['ANNO_FINE_STESURA'], None]

opere_autori_generi_testi['ANNO_RIFERIMENTO'] = np.select(conditions, values, default=None)
opere_autori_generi_testi['ANNO_RIFERIMENTO'] = opere_autori_generi_testi['ANNO_RIFERIMENTO'].astype('Int64')
# sum(~opere_autori_generi_testi['ANNO_RIFERIMENTO'].notnull()) nessun null, abbiamo una data di riferimento per ognuno

- Il libro più antico ha una fine stesura nel 1224
- Ottengo una suddivisione della storia italiana da varie fonti:
  - https://it.wikipedia.org/wiki/Storia_d%27Italia#Domini_spagnoli_in_Italia
- Genero un nuovo campo 'Periodo_Storico' che userò per suddividere
- Genero un nuovo campo 'Info_Periodo_Storico'

La generazione avviene SIA sulle opere di SCRIPTA che su quelle AGGIUNTIVE

In [64]:
# Range periodi storici italia
range_periodi_storici_italia = [1000, 1250, 1454, 1559, 1707, 1796, 1814, 1860, 1914, 1922, 1945]
# Label da assegnare
range_periodi_storici_italia_label = ['1000_1250_Pieno Medievo', # 5 testi
                                '1250_1454_Tardo_Medioevo', #113 testi
                                '1454_1559_Rinascimento', #229 testi
                                '1559_1707_Seicento', #267 ttesti 
                                '1707_1796_Settecento', #322 testi
                                '1796_1814_Periodo_napoleonico', #90 testi
                                '1814_1860_Risorgimento', #361 opere
                                '1861_1914_L\'Italia_liberale', #1352
                                '1915_1922_L\'Italia_nella_Prima_Guerra_Mondiale_e_dopoguerra', # 308
                                '1922_1945_Ventennio_Fascista_Seconda_Guerra_Mondiale', # 11 romanzi
                                '1945_presente_La_Repubblica_Italiana' # 61 romanzi
                                ]
range_periodi_storici_italia_meta = ['Lotta per le investiture, Crociate, Normanni e regno di sicilia, I comuni, Federico II di Svevia',
    'Evoluzione comuni nord italia in signorie Declino Impero e papato, Republiche Marinare, Angioini e Aragonesi nel meridione, Lotte tra stati italiani',
    'Pace di lodi (equilibrio) e sviluppo rinascimento/umanesimo, Scoperta America, Guerre D\'italia e Riforma Protestante, Carlo VIII, controriforma e concilio di Trento, Carlo V, Venezia guerre con impero ottomano',
    'Pace di Cateau-Cambrésis 1559, Domini spagnoli in Italia, Guerra di successione spagnola con successione austriaca 1707',
    'Dominazione austriaca degli Asburgo, guerra di successione Polacca e rottenimento dei regni di Napoli e di Siciali dei Borbone di Spagna, Illuminismo',
    'Periodo napoleonico',
    'La Restaurazione (1815-1848) - Regno borbonico delle due Sicilie - Il Regno di Sardegna, I moti carbonari (prima e seconda guerra d\'indipendenza',
    'Destra storica, Sinistra storica, Questione meridionale',
    'L\'iniziale neutralità, 1915-1918, epilogo',
    'Marcia su Roma, Dittatura e impero, Allenza con la Germania, Caduta e resisttenza, Epilogo e costo della guerra',
    'Gli anni costituenti, La prima repubblica, La seconda repubblica']

# Opere da SCRIPTA
conditions = [
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[1])).to_numpy(dtype=bool), 
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[1]) & (opere_autori_generi_testi['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[2])).to_numpy(dtype=bool),
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[2]) & (opere_autori_generi_testi['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[3])).to_numpy(dtype=bool),
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[3]) & (opere_autori_generi_testi['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[4])).to_numpy(dtype=bool),
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[4]) & (opere_autori_generi_testi['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[5])).to_numpy(dtype=bool),
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[5]) & (opere_autori_generi_testi['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[6])).to_numpy(dtype=bool),
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[6]) & (opere_autori_generi_testi['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[7])).to_numpy(dtype=bool),
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[7]) & (opere_autori_generi_testi['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[8])).to_numpy(dtype=bool),
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[8]) & (opere_autori_generi_testi['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[9])).to_numpy(dtype=bool),
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[9]) & (opere_autori_generi_testi['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[10])).to_numpy(dtype=bool),
    ((opere_autori_generi_testi['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[10])).to_numpy(dtype=bool)
    ]

# Opere aggiuntive
conditions1 = [
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[1])).to_numpy(dtype=bool), 
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[1]) & (opere_aggiuntive['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[2])).to_numpy(dtype=bool),
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[2]) & (opere_aggiuntive['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[3])).to_numpy(dtype=bool),
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[3]) & (opere_aggiuntive['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[4])).to_numpy(dtype=bool),
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[4]) & (opere_aggiuntive['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[5])).to_numpy(dtype=bool),
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[5]) & (opere_aggiuntive['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[6])).to_numpy(dtype=bool),
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[6]) & (opere_aggiuntive['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[7])).to_numpy(dtype=bool),
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[7]) & (opere_aggiuntive['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[8])).to_numpy(dtype=bool),
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[8]) & (opere_aggiuntive['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[9])).to_numpy(dtype=bool),
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[9]) & (opere_aggiuntive['ANNO_RIFERIMENTO'] < range_periodi_storici_italia[10])).to_numpy(dtype=bool),
    ((opere_aggiuntive['ANNO_RIFERIMENTO'] >= range_periodi_storici_italia[10])).to_numpy(dtype=bool)
    ]

# Opere da SCRIPTA
opere_autori_generi_testi['PERIODO_STORICO'] = np.select(conditions, range_periodi_storici_italia_label, default=None)
opere_autori_generi_testi['INFO_PERIODO_STORICO'] = np.select(conditions, range_periodi_storici_italia_meta, default=None)
# Opere Aggiuntive
opere_aggiuntive['PERIODO_STORICO'] = np.select(conditions1, range_periodi_storici_italia_label, default=None)
opere_aggiuntive['INFO_PERIODO_STORICO'] = np.select(conditions1, range_periodi_storici_italia_meta, default=None)

In [65]:
# check correttezza assegnazioni
opere_autori_generi_testi[['ANNO_RIFERIMENTO', 'PERIODO_STORICO', 'INFO_PERIODO_STORICO']].head(10)

,ANNO_RIFERIMENTO,PERIODO_STORICO,INFO_PERIODO_STORICO
0,1532,1454_1559_Rinascimento,Pace di lodi (equilibrio) e sviluppo rinascime...
1,1554,1454_1559_Rinascimento,Pace di lodi (equilibrio) e sviluppo rinascime...
2,1353,1250_1454_Tardo_Medioevo,Evoluzione comuni nord italia in signorie Decl...
3,1494,1454_1559_Rinascimento,Pace di lodi (equilibrio) e sviluppo rinascime...
4,1838,1814_1860_Risorgimento,La Restaurazione (1815-1848) - Regno borbonico...
5,1901,1861_1914_L'Italia_liberale,"Destra storica, Sinistra storica, Questione me..."
6,1522,1454_1559_Rinascimento,Pace di lodi (equilibrio) e sviluppo rinascime...
7,1913,1861_1914_L'Italia_liberale,"Destra storica, Sinistra storica, Questione me..."
8,1890,1861_1914_L'Italia_liberale,"Destra storica, Sinistra storica, Questione me..."
9,1894,1861_1914_L'Italia_liberale,"Destra storica, Sinistra storica, Questione me..."


In [66]:
opere_aggiuntive[['ANNO_RIFERIMENTO', 'PERIODO_STORICO', 'INFO_PERIODO_STORICO']].head(10)

,ANNO_RIFERIMENTO,PERIODO_STORICO,INFO_PERIODO_STORICO
0,1923,1922_1945_Ventennio_Fascista_Seconda_Guerra_Mo...,"Marcia su Roma, Dittatura e impero, Allenza co..."
1,1926,1922_1945_Ventennio_Fascista_Seconda_Guerra_Mo...,"Marcia su Roma, Dittatura e impero, Allenza co..."
2,1930,1922_1945_Ventennio_Fascista_Seconda_Guerra_Mo...,"Marcia su Roma, Dittatura e impero, Allenza co..."
3,1933,1922_1945_Ventennio_Fascista_Seconda_Guerra_Mo...,"Marcia su Roma, Dittatura e impero, Allenza co..."
4,1934,1922_1945_Ventennio_Fascista_Seconda_Guerra_Mo...,"Marcia su Roma, Dittatura e impero, Allenza co..."
5,1937,1922_1945_Ventennio_Fascista_Seconda_Guerra_Mo...,"Marcia su Roma, Dittatura e impero, Allenza co..."
6,1938,1922_1945_Ventennio_Fascista_Seconda_Guerra_Mo...,"Marcia su Roma, Dittatura e impero, Allenza co..."
7,1939,1922_1945_Ventennio_Fascista_Seconda_Guerra_Mo...,"Marcia su Roma, Dittatura e impero, Allenza co..."
8,1940,1922_1945_Ventennio_Fascista_Seconda_Guerra_Mo...,"Marcia su Roma, Dittatura e impero, Allenza co..."
9,1944,1922_1945_Ventennio_Fascista_Seconda_Guerra_Mo...,"Marcia su Roma, Dittatura e impero, Allenza co..."


### Generazione corpus 

Utilizzo una funzione analoga a quella vista per rigenerare i libri, ma qui per generare i corpus
- Accortezza di aggiungere i libri post 2022 nelle due epoche

In [133]:
# Check che tutti i libri e i vocaboli abbiano id non nullo
sum(~vocabolario['ID_OPERA'].notnull())
sum(~opere_autori_generi_testi['ID_OPERA'].notnull())

0

In [153]:
path_libri_trattati = '.\data\\books_after_1922\\libri_trattati'


# passare dataframe completo
def build_corpus_periodo_storico(vocabolario_, meta_data, path):
    label_periodo_storico = meta_data['PERIODO_STORICO'].unique(
    ).tolist()  # estraggo periodi storici
    for label in label_periodo_storico:
        corpus_title = label  # nome del corpus

        if ((label != '1922_1945_Ventennio_Fascista_Seconda_Guerra_Mondiale') & (label != '1945_presente_La_Repubblica_Italiana')):
            # opere relative al periodo storico
            opere_periodo = meta_data[meta_data['PERIODO_STORICO'] == label]
            lista_opere_periodo = opere_periodo['ID_OPERA'].unique().tolist()

            vocabolario_periodo = vocabolario_[vocabolario_['ID_OPERA'].isin(
                lista_opere_periodo)]  # selezione vocaboli delle opere in questione
            print(label)
            print(str(len(opere_periodo)) + ' opere')
            try:
                with open(path + corpus_title + '.txt', "r", encoding='utf-8') as file:
                    # print('pass')
                    pass
            except FileNotFoundError:
                # print(vocabolario_periodo.head(5))

                # Sostituisco eventuali NaN con la sigla MW (Missing Word) o MS (Missing Separator)
                vocabolario_periodo.VOCABOLO = vocabolario_periodo.VOCABOLO.fillna(
                    'MW')
                vocabolario_periodo.SEPARATORE = vocabolario_periodo.SEPARATORE.fillna(
                    'MS')

                # Traduco le lettere nei separatori
                vocabolario_periodo['SEPARATORE'] = vocabolario_periodo['SEPARATORE'].str.replace(
                    'S', ' ', regex=False)
                vocabolario_periodo['SEPARATORE'] = vocabolario_periodo['SEPARATORE'].str.replace(
                    'P', '\n', regex=False)  # uso \n per andare a capo quando scrivo il .txt

                # Creo la lista con VOCABOLO + SEPARATORE
                word_list = vocabolario_periodo['VOCABOLO']
                sep_list = vocabolario_periodo['SEPARATORE']
                result = [None]*(len(word_list)+len(sep_list))
                result[::2] = word_list
                result[1::2] = sep_list

                # Creo il dataframe necessario, con manipolazione
                df_2 = pd.DataFrame(result)
                df_2.rename(columns={0: "PAROLA"}, inplace=True)
                df_2['PARAGRAFO'] = df_2["PAROLA"].str.contains(
                    '\n', regex=False).cumsum()
                df_2['PARAGRAFO'] = df_2['PARAGRAFO'].where(
                    ~df_2['PAROLA'].str.contains('\n', regex=False), df_2['PARAGRAFO']-1)
                # Riduco di 1 il valore di paragrafo quando ho l'escape character \n per andare a capo nel punto giusto

                # Creo il dataframe per costruire le frasi, con manipolazione
                groups = df_2.groupby("PARAGRAFO")
                sentences = groups.apply(lambda sdf: "".join(sdf.PAROLA))
                df_3 = pd.DataFrame(sentences)
                df_3.reset_index(inplace=True)
                df_3.drop('PARAGRAFO', axis=1, inplace=True)

                with open(path + corpus_title + '.txt', "w", encoding='utf-8') as file:
                    # Testo
                    for index, row in df_3.iterrows():
                        file.write(row[0])
            finally:
                pass
        # unisco libri aggiuntivi fino al 1945
        elif (label == '1922_1945_Ventennio_Fascista_Seconda_Guerra_Mondiale'):
            print(label)
            try:
                with open(path + corpus_title + '.txt', "r", encoding='utf-8') as file:
                    # print('pass')
                    pass
            except FileNotFoundError:
                with open(path + corpus_title + '.txt', "w", encoding='utf-8') as file:
                    # Testo
                    # for index, row in df_3.iterrows():
                    #    file.write(row[0])
                    print(
                        str(len(os.listdir(f"{path_libri_trattati}\pre_1945"))) + ' opere')
                    for filename in os.listdir(f"{path_libri_trattati}\pre_1945"):
                        filedata = None  # azzero
                        with open(f"{path_libri_trattati}\pre_1945\{filename}", "r", encoding="utf8") as file_book:
                            # filedata = file_book.read()
                            for line in file_book:
                                file.write(line)
            finally:
                pass
        else:  # (label == '1945_presente_La_Repubblica_Italiana'): #unisco libri aggiuntivi dopo 1945
            try:
                with open(path + corpus_title + '.txt', "r", encoding='utf-8') as file:
                    # print('pass')
                    pass
            except FileNotFoundError:
                with open(path + corpus_title + '.txt', "w", encoding='utf-8') as file:
                    # Testo
                    # for index, row in df_3.iterrows():
                    #     file.write(row[0])
                    print(
                        str(len(os.listdir(f"{path_libri_trattati}\post_1945"))) + ' opere')
                    for filename in os.listdir(f"{path_libri_trattati}\post_1945"):
                        filedata = None  # azzero
                        with open(f"{path_libri_trattati}\post_1945\{filename}", "r", encoding="utf8") as file_book:
                            # filedata = file_book.read()
                            for line in file_book:
                                file.write(line)
            finally:
                pass


In [154]:
build_corpus_periodo_storico(
    vocabolario, opere_autori_generi_testi, "./data/Corpora/periodi_storici/")


1454_1559_Rinascimento
229 opere
1250_1454_Tardo_Medioevo
113 opere
1814_1860_Risorgimento
361 opere
1861_1914_L'Italia_liberale
1352 opere
1559_1707_Seicento
267 opere
1915_1922_L'Italia_nella_Prima_Guerra_Mondiale_e_dopoguerra
308 opere
1796_1814_Periodo_napoleonico
90 opere
1707_1796_Settecento
322 opere
1000_1250_Pieno Medievo
5 opere
1922_1945_Ventennio_Fascista_Seconda_Guerra_Mondiale
11 opere
61 opere


## Divisione per movimenti letterali

Fonti:
- https://it.wikipedia.org/wiki/Storia_della_letteratura_italiana#La_letteratura_religiosa
- https://it.wikipedia.org/wiki/Corrente_letteraria

Definisco quelle che sono le diverse correnti letterarie e i principali esponenti, al fine poi di andare a creare una label che assegni a ogni opera una label.

### Definizione periodi

In [29]:
#################### Duecento: 16 opere ####################

# Duecento poetica: 6 opere
duecento_letteratura_allegorica = [
    'Alano di Lilla', 'Giacomino da Verona', 'Bonvesin de la Riva degli Umiliati']  # 0 opere
duecento_letteratura_religiosa = ['Francesco d\'Assisi', 'Raniero Fasani',
                                  'Tommaso da Celano', 'Bonaventura da Bagnoregio', 'Jacopone da Todi']  # due opere
duecento_scuola_siciliana = ['Giacomo da Lentini', 'Guido delle Colonne', 'Pier della Vigna',
                             'Rinaldo d\'Aquino', 'Giacomo Pugliese', 'Stefano Protonotaro', 'Cielo d\'Alcamo']  # due opere
duecento_scuola_toscana = ['Bonagiunta Orbicciani da Lucca', 'Monte Andrea',
                           'Chiaro Davanzati', 'Compiuta Donzella', 'Guittone']  # una opera
duecento_peosia_comico_realistica = ['Rustico di Filippo', 'Meo de\' Tolomei',
                                     'Cenne della Chitarra', 'Cecco Angiolieri', 'Folgóre da San Gimignano']  # una opera
duecento_poetica = duecento_letteratura_allegorica + duecento_letteratura_religiosa + \
    duecento_scuola_siciliana + duecento_scuola_toscana + \
    duecento_peosia_comico_realistica

# Duecento prosa: 10 opere
duecento_prosa = ['Salimbene de Adam', 'Jacopo da Varazze', 'Brunetto Latini', 'Bono Giamboni',
                  'Restoro d\'Arezzo', 'Paolino Minorita', 'Guidotto da Bologna', 'Ricordano Malispini', 'Marco Polo', 'Rustichello da Pisa']  # 10 opere

duecento_poetica_e_prosa = duecento_poetica + duecento_prosa  # 16 opere

#################### Trecento ####################

# DOLCE STIL NOVO :10 opere

dolce_stil_novo = ['Dante Alighieri', 'Guido Guinizzelli', 'Guido Cavalcanti',
                   'Lapo Gianni', 'Gianni Alfani', 'Cino da Pistoia', 'Dino Frescobaldi']  # 10 opere

# PETRARCHISMO

# petrarca è considerato il più grande PRECURSORE dell'umanesimo, si può includere nel punto dopo
petrarchismo = ['Francesco Petrarca', 'Boiardo', 'Poliziano', 'Lorenzo il Magnifico', 'Michelangelo Buonarroti', 'Alfonso Gatto', 'Libero de Libero', 'Mario Luzi', 'Leonardo Sinisgalli',
                'Corrado Pavolini', 'Gatto', 'Piero Bigongiari', 'Sinisgalli', 'Luzi', 'Alessandro Parronchi', 'Vittorio Sereni']  # 6 opere

#DolceStilNovo_petrarchismo = dolce_stil_novo + petrarchismo


#################### Quattrocento e prima metà Cinquecento (RINASCIMENTO)): Umanesimo, letteratura volgare, letteratura rinascimentale ####################

# UMANESIMO

# 14 opere, precursore, insieme a petrarca, dell'umanesimo, egli è ancora poeta del trecento
boccaccio = ['Giovanni Boccaccio', 'Giambattista Basile']

umanesimo = ['Ludovico Ariosto', 'Francesco Guicciardini', ' Tommaso Campanella', 'Leonardo Bruni', 'Poggio Bracciolini', 'Lorenzo Valla', 'Flavio Biondo', 'Leon Battista Alberti', 'Marsilio Ficino',
             'Cristoforo Landino', 'Pico della Mirandola', 'Poliziano', 'Savonarola', 'Leonardo da Vinci', 'Machiavelli', 'Feo Belcari', 'Bernardino da Siena']

# torquato_tasso = ['Torquato Tasso'] # 10 opere, non si può inserirlo nell'umanesimo perchè a cavallo tra rinascimento e controriforma (influenze diverse)

boccaccio_e_umanesimo = umanesimo + boccaccio  # 92 opere

#################### Seicento ####################

# MANIERISMO e LETTERATURA BAROCCA (consecutivi e molto simili nella visione del mondo, per questo li accorpo)

# Segna il passaggio tra la cultura rinascimentale e quella barocca
manierismo = ['Torquato Tasso', 'Giordano Bruno', 'Battista Guarini']  # 17

barocco = ['Giambattista Marino', 'Alessandro Tassoni',
           'Gabriello Chiabrera', 'Fulvio Testi', 'Galileo Galilei', 'Copernico']  # 30

manirismo = ["Girolamo Fontanella", "Scipione Errico", "Girolamo Brusoni", "Giacomo Lubrano", "Giambattista Basile",
             "Girolamo Preti", "Francesco Bracciolini", "Guidubaldo Benamati", "Agazio di Somma", "Scipione Errico"]  # 11

manierismo_e_barocco = manierismo + barocco + manirismo

#################### Settecento (e non solo) ####################

# Classicismo (ARCADIA) e Neoclassicismo

# ARCADIA (classicismo anti-barocco, si ispira al petrarchismo) -> 63 opere

classicismo_arcadia = ['Pietro Metastasio', 'Gian Vincenzo Gravina',
                       'Giambattista Felice Zappi', 'Paolo Rolli']  # 63 opere

# NEOCLASSICISMO (non lo trattiamo perchè si mischia con illuminismo e rimanticismo)
# neoclassicismo = ["Ludovico Savioli", "Giuseppe Parini",
#                  "Vincenzo Monti", "Vittorio Alfieri", "Ugo Foscolo"] # 60 opere

# ILLUMINISMO e NEOCLASSICISMO -> 61 opere

illuminismo_neoclassicismo = ['Cesare Beccaria', 'Pietro Verri', 'Giuseppe Parini', 'Vittorio Alfieri', 'Ludovico Savioli', "Francesco Algarotti", "Maria Gaetana Agnesi",
                              "Sallustio Bandini", "Giuseppe Baretti", "Melchiorre Cesarotti", "Carlo Denina", "Gaetano Filangieri", "Antonio Genovesi", "Pietro Giannone", "Gaspare Gozzi", "Alessandro Verri"]

#################### Fine settecento e Ottocento ####################

# PRE-ROMANTICISMO e ROMANTICISMO -> 78 opere

romanticismo = ["Ugo Foscolo", "Ippolito Pindemonte", "Alessandro Manzoni", "Giacomo Leopardi",
                "Giuseppe Botero", "Ippolito Nievo", "Massimo d'Azeglio", "Carlo Pisacane", "Vincenzo Gioberti",
                "Giuseppe Mazzini", "Massimo d'Azeglio", "Vincenzo Gioberti", "Luigi Settembrini", "Carlo Cattaneo", "Carlo Pisacane", "Cesare Cantù", "Gino Capponi", "Vincenzo Cuoco", "Silvio Pellico"]

verismo = ["Giovanni Verga", "Luigi Capuana", "Federico De Roberto", "Matilde Serao", "Salvatore Di Giacomo",
           "Nicola Misasi", "Grazia Deledda", "Domenico Ciampoli", "Giuseppe Mezzanotte", "Cesare Pascarella", "Renato Fucini"]

decadentismo = ["Giovanni Pascoli", "Italo Svevo", "Antonio Fogazzaro",
                "Luigi Pirandello", "Gabriele D'Annunzio", "Giuseppe Antonio Borghese"]

#################### PRIMO NOVECENTO ####################

# AVANGUARDIE del primo novecento

crepuscolarismo = ["Corrado Govoni", "Marino Moretti",
                   "Sergio Corazzini", "Guido Gozzano"]
futurismo = ["Filippo Tommaso Marinetti", "Aldo Palazzeschi",
             "Francesco Cangiullo", "Luciano Folgore", "Ardengo Soffici", "Corrado Govoni"]

ermetismo = ["Camillo Sbarbaro", "Clemente Rebora", "Dino Campana", "Arturo Onofri",
             "Giuseppe Ungaretti", "Salvatore Quasimodo", "Alfonso Gatto", "Vittorio Sereni", "Mario Luzi"]

avanguardie_primo_novecento = crepuscolarismo + futurismo + ermetismo

#################### DOPOGUERRA ####################

# Neorealismo

neorealismo = ["Alberto Moravia", "Ignazio Silone", "Vasco Pratolini", "Beppe Fenoglio", "Pier Paolo Pasolini", "Carlo Cassola", "Giorgio Bassani", "Vasco Pratolini", "Carlo Emilio Gadda", "Emilio Lussu", "Mario Rigoni Stern", "Italo Calvino", "Dino Buzzati", "Carlo Levi", "Alberto Moravia", "Elsa Morante", "Cesare Pavese", "Ignazio Silone", "Ennio Flaiano", "Goffredo Parise", "Pier Paolo Pasolini", "Primo Levi", "Carlo Bernari", "Corrado Alvaro", "Guido Piovene", "Rocco Scotellaro", "Vitaliano Brancati", "Giuseppe Tomasi di Lampedusa", "Elio Vittorini",
               "Leonardo Sciascia", "Gesualdo Bufalino", "Romano Bilenchi", "Natalia Ginzburg", "Giovanni Arpino", "Umberto Eco", "Beppe Fenoglio", "Antonio Tabucchi", "Giuseppe Berto", "Giovanni Testori", "Pier Antonio Quarantotti Gambini", "Francesco Jovine", "Antonio Delfini", "Giovannino Guareschi", "Alberto Arbasino", "Luciano Bianciardi", "Attilio Bertolucci", "Giorgio Caproni", "Sandro Penna", "Vittorio Sereni", "Giorgio Orelli", "Mario Soldati", "Lucio Mastronardi", "Luigi Malerba", "Antonio Pizzuto", "Dacia Maraini", "Tommaso Landolfi", "Vincenzo Consolo", "Andrea Camilleri", "Eugenio Corti"]


- duecento_poetica_e_prosa (16 opere)  
- dolce_stil_novo (10 opere)
- petrarchismo (6 opere)
- boccaccio_e_umanesimo (92)
- manierismo_e_barocco (58)
- classicismo_arcadia (63 opere)
- illuminismo_neoclassicismo (61 opere)
- romanticismo (112)
- verismo (171)
- decadentismo (108 + 3 libri aggiuntivi)
- avanguardie_primo_novecento (24 + 1 libro aggiuntivo)
- neorealismo (42 libri aggiuntivi)

Autori importanti: Dante Alighieri, Francesco Petrarca, Giovanni Boccaccio, Ludovico Ariosto, Torquato Tasso, Giambattista Marino, Giuseppe Parini, Vittorio Alfieri (preromantico), Ugo Foscolo (preromantico), Alessandro Manzoni (romantico), Giacomo Leopardi (romanttico differentte)

Check numero di opere in ogni periodo

In [59]:
# opere_autori_generi_testi[opere_autori_generi_testi['AUTORE'].str.contains("Dino Frescobaldi", na=False)]
# opere_autori_generi_testi[opere_autori_generi_testi['AUTORE'] in dolce_stil_novo]

print(len(opere_autori_generi_testi[opere_autori_generi_testi['AUTORE'].str.contains("|".join(neorealismo), na=False, case = False)]))
opere_autori_generi_testi[opere_autori_generi_testi['AUTORE'].str.contains("|".join(neorealismo), na=False, case = False)].head(10)

# opere aggiuntive
print(len(opere_aggiuntive[opere_aggiuntive['AUTORE'].str.contains("|".join(verismo), na=False, case = False)]))
opere_aggiuntive[opere_aggiuntive['AUTORE'].str.contains("|".join(verismo), na=False, case = False)].head(10)

1
0


,ANNO_RIFERIMENTO,TITOLO,AUTORE,Filename


In [ ]:
# Check che le opere con autore null non rompano tutto
# opere_autori_generi_testi[~opere_autori_generi_testi['AUTORE'].notnull()]['AUTORE'].str.contains("|".join(verismo), na=False, case = False)

### Generazione colonna FASE_LETTERARIA

In [75]:
# Range periodi storici italia
fasi_letterarie_italiane_liste_autori = [duecento_poetica_e_prosa, dolce_stil_novo, petrarchismo,
                                boccaccio_e_umanesimo, manierismo_e_barocco, classicismo_arcadia, 
                                illuminismo_neoclassicismo, romanticismo, verismo, decadentismo, avanguardie_primo_novecento, neorealismo]
# Label da assegnare
fasi_letterarie_italiane = ['1_duecento_poetica_e_prosa Medievo',
                            '2_dolce_stil_novo',
                            '3_petrarchismo',
                            '4_boccaccio_e_umanesimo',
                            '5_manierismo_e_barocco',
                            '6_classicismo_arcadia',
                            '7_illuminismo_neoclassicismo',
                            '8_romanticismo',
                            '9_verismo',
                            '10_decadentismo',
                            '11_avanguardie_primo_novecento',
                            '12_neorealismo'
                            ]

# Opere da SCRIPTA
conditions = [
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[0]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[1]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[2]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[3]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[4]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[5]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[6]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[7]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[8]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[9]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[10]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_autori_generi_testi['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[11]), na=False, case = False)).to_numpy(dtype=bool),
]

# Opere aggiuntive
conditions1 = [
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[0]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[1]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[2]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[3]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[4]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[5]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[6]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[7]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[8]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[9]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[10]), na=False, case = False)).to_numpy(dtype=bool),
    (opere_aggiuntive['AUTORE'].str.contains("|".join(fasi_letterarie_italiane_liste_autori[11]), na=False, case = False)).to_numpy(dtype=bool),
]

# Opere da SCRIPTA
opere_autori_generi_testi['FASE_LETTERARIA'] = np.select(
    conditions, fasi_letterarie_italiane, default=None)
# Opere Aggiuntive
opere_aggiuntive['FASE_LETTERARIA'] = np.select(
    conditions1, fasi_letterarie_italiane, default=None)

In [88]:
# Check dei not null e print
print(sum(opere_autori_generi_testi['FASE_LETTERARIA'].notnull())) # conti tornano più o meno
print(opere_autori_generi_testi[['AUTORE', 'FASE_LETTERARIA']].head(5))
opere_autori_generi_testi['FASE_LETTERARIA'].value_counts(dropna=False)

719
                 AUTORE          FASE_LETTERARIA
0      Ludovico Ariosto  4_boccaccio_e_umanesimo
1       Matteo Bandello                     None
2    Giovanni Boccaccio  4_boccaccio_e_umanesimo
3  Matteo Maria Boiardo           3_petrarchismo
4          Cesare Cantù           8_romanticismo


None                                  2392
9_verismo                              171
8_romanticismo                         112
10_decadentismo                        108
4_boccaccio_e_umanesimo                 90
6_classicismo_arcadia                   63
7_illuminismo_neoclassicismo            61
5_manierismo_e_barocco                  57
11_avanguardie_primo_novecento          24
1_duecento_poetica_e_prosa Medievo      16
2_dolce_stil_novo                       10
3_petrarchismo                           6
12_neorealismo                           1
Name: FASE_LETTERARIA, dtype: int64

In [89]:
print(sum(opere_aggiuntive['FASE_LETTERARIA'].notnull())) # conti tornano
print(opere_aggiuntive[['AUTORE', 'FASE_LETTERARIA']].tail(5))
opere_aggiuntive['FASE_LETTERARIA'].value_counts(dropna=False)

46
                 AUTORE FASE_LETTERARIA
67  Pier Paolo Pasolini  12_neorealismo
68     Antonio Tabucchi  12_neorealismo
69    Goliarda Sapienza            None
70          Umberto Eco  12_neorealismo
71       Elena Ferrante            None


12_neorealismo                    42
None                              26
10_decadentismo                    3
11_avanguardie_primo_novecento     1
Name: FASE_LETTERARIA, dtype: int64

In [96]:
"10_decadentismo" not in opere_aggiuntive['FASE_LETTERARIA'].dropna().unique().tolist()

False

### Generazione corpus

In [106]:
path_libri_trattati = '.\data\\books_after_1922\\libri_trattati\\tutti'


# passare dataframe completo
def build_corpus_fasi_letterarie(vocabolario_, meta_data, meta_data_agg, path): 
    for label in fasi_letterarie_italiane: # estraggo periodi storici
        corpus_title = label  # nome del corpus
        
        # opere relative alla fase specifica
        opere_fase = meta_data[meta_data['FASE_LETTERARIA'] == label]
        lista_opere_fase = opere_fase['ID_OPERA'].unique().tolist()

        vocabolario_fase = vocabolario_[vocabolario_['ID_OPERA'].isin(
            lista_opere_fase)]  # selezione vocaboli delle opere in questione
        print(label + "\t" + str(len(opere_fase) + len(meta_data_agg[meta_data_agg['FASE_LETTERARIA'] == label])) + ' opere totali')
        try:
            with open(path + corpus_title + '.txt', "r", encoding='utf-8') as file:
                print('---')
                pass
        except FileNotFoundError:
            # print(vocabolario_fase.head(5))
            print('-->')
            # Sostituisco eventuali NaN con la sigla MW (Missing Word) o MS (Missing Separator)
            vocabolario_fase.VOCABOLO = vocabolario_fase.VOCABOLO.fillna(
                'MW')
            vocabolario_fase.SEPARATORE = vocabolario_fase.SEPARATORE.fillna(
                'MS')

            # Traduco le lettere nei separatori
            vocabolario_fase['SEPARATORE'] = vocabolario_fase['SEPARATORE'].str.replace(
                'S', ' ', regex=False)
            vocabolario_fase['SEPARATORE'] = vocabolario_fase['SEPARATORE'].str.replace(
                'P', '\n', regex=False)  # uso \n per andare a capo quando scrivo il .txt

            # Creo la lista con VOCABOLO + SEPARATORE
            word_list = vocabolario_fase['VOCABOLO']
            sep_list = vocabolario_fase['SEPARATORE']
            result = [None]*(len(word_list)+len(sep_list))
            result[::2] = word_list
            result[1::2] = sep_list

            # Creo il dataframe necessario, con manipolazione
            df_2 = pd.DataFrame(result)
            df_2.rename(columns={0: "PAROLA"}, inplace=True)
            df_2['PARAGRAFO'] = df_2["PAROLA"].str.contains(
                '\n', regex=False).cumsum()
            df_2['PARAGRAFO'] = df_2['PARAGRAFO'].where(
                ~df_2['PAROLA'].str.contains('\n', regex=False), df_2['PARAGRAFO']-1)
            # Riduco di 1 il valore di paragrafo quando ho l'escape character \n per andare a capo nel punto giusto

            # Creo il dataframe per costruire le frasi, con manipolazione
            groups = df_2.groupby("PARAGRAFO")
            sentences = groups.apply(lambda sdf: "".join(sdf.PAROLA))
            df_3 = pd.DataFrame(sentences)
            df_3.reset_index(inplace=True)
            df_3.drop('PARAGRAFO', axis=1, inplace=True)

            with open(path + corpus_title + '.txt', "w", encoding='utf-8') as file:
                # Testo
                for index, row in df_3.iterrows():
                    file.write(row[0])
                # Se e solo se ci sono libri aggiuntivi per quella fase letteraria procedo aggiungendoli
                if (label in meta_data_agg['FASE_LETTERARIA'].dropna().unique().tolist()):
                    # Dalle opere aggiuntive seleziono solo le rilevanti 
                    opere_fase = meta_data_agg[meta_data_agg['FASE_LETTERARIA'] == label]
                    # Selezioni i nomi dei file unici
                    lista_path_opere_fase = opere_fase['Filename'].unique().tolist()
                    # Seleziono uno a uno i diversi file e li aggiungo
                    for filename in lista_path_opere_fase:
                        filedata = None  # azzero
                        with open(f"{path_libri_trattati}\{filename}", "r", encoding="utf8") as file_book:
                            # filedata = file_book.read()
                            for line in file_book:
                                file.write(line)        
        finally:
            pass


In [107]:
output_corpora_fasi_letterarie = "./data/Corpora/fasi_letterarie/"

build_corpus_fasi_letterarie(
    vocabolario, opere_autori_generi_testi, opere_aggiuntive, output_corpora_fasi_letterarie)

1_duecento_poetica_e_prosa Medievo	16 opere totali
---
2_dolce_stil_novo	10 opere totali
---
3_petrarchismo	6 opere totali
---
4_boccaccio_e_umanesimo	90 opere totali
---
5_manierismo_e_barocco	57 opere totali
---
6_classicismo_arcadia	63 opere totali
---
7_illuminismo_neoclassicismo	61 opere totali
---
8_romanticismo	112 opere totali
---
9_verismo	171 opere totali
---
10_decadentismo	111 opere totali
---
11_avanguardie_primo_novecento	25 opere totali
---
12_neorealismo	43 opere totali
---


## Divisione per Autore

Del tutto analogo il procedimento al precedente

In [22]:
autori_selezionati = ["Dante Alighieri", "Francesco Petrarca", "Giovanni Boccaccio", "Ludovico Ariosto", "Torquato Tasso", "Giuseppe Parini", "Vittorio Alfieri", "Ugo Foscolo", "Alessandro Manzoni",
                      "Giacomo Leopardi", "Italo Svevo", "Luigi Pirandello", "Gabriele D'Annunzio", "Pier Paolo Pasolini", "Italo Calvino", "Dino Buzzati", "Primo Levi", "Umberto Eco"]


In [20]:
path_libri_trattati = '.\data\\books_after_1922\\libri_trattati\\tutti'


# passare dataframe completo
def build_corpus_autori(vocabolario_, meta_data, meta_data_agg, path): 
    for label in autori_selezionati: # estraggo periodi storici
        corpus_title = label  # nome del corpus
        # opere relative all'autore specifico
        opere_autore = meta_data[meta_data['AUTORE'].str.contains(label, na=False, case = False)]
        lista_opere_autore = opere_autore['ID_OPERA'].unique().tolist()

        vocabolario_autore = vocabolario_[vocabolario_['ID_OPERA'].isin(
            lista_opere_autore)]  # selezione vocaboli delle opere in questione
        print(label + "\t" + str(len(opere_autore) + len(meta_data_agg[meta_data_agg['AUTORE'].str.contains(label, na=False, case = False)])) + ' opere totali')
        try:
            with open(path + corpus_title + '.txt', "r", encoding='utf-8') as file:
                print('---')
                pass
        except FileNotFoundError:
            # print(vocabolario_autore.head(5))
            print('-->')
            # Sostituisco eventuali NaN con la sigla MW (Missing Word) o MS (Missing Separator)
            if (len(opere_autore) != 0): # check che ci sia almeno un libro in SCRIPTA (altrimenti da errore)
                vocabolario_autore.VOCABOLO = vocabolario_autore.VOCABOLO.fillna(
                    'MW')
                vocabolario_autore.SEPARATORE = vocabolario_autore.SEPARATORE.fillna(
                    'MS')

                # Traduco le lettere nei separatori
                vocabolario_autore['SEPARATORE'] = vocabolario_autore['SEPARATORE'].str.replace(
                    'S', ' ', regex=False)
                vocabolario_autore['SEPARATORE'] = vocabolario_autore['SEPARATORE'].str.replace(
                    'P', '\n', regex=False)  # uso \n per andare a capo quando scrivo il .txt

                # Creo la lista con VOCABOLO + SEPARATORE
                word_list = vocabolario_autore['VOCABOLO']
                sep_list = vocabolario_autore['SEPARATORE']
                result = [None]*(len(word_list)+len(sep_list))
                result[::2] = word_list
                result[1::2] = sep_list

                # Creo il dataframe necessario, con manipolazione
                df_2 = pd.DataFrame(result)
                df_2.rename(columns={0: "PAROLA"}, inplace=True)
                df_2['PARAGRAFO'] = df_2["PAROLA"].str.contains(
                    '\n', regex=False).cumsum()
                df_2['PARAGRAFO'] = df_2['PARAGRAFO'].where(
                    ~df_2['PAROLA'].str.contains('\n', regex=False), df_2['PARAGRAFO']-1)
                # Riduco di 1 il valore di paragrafo quando ho l'escape character \n per andare a capo nel punto giusto

                # Creo il dataframe per costruire le frasi, con manipolazione
                groups = df_2.groupby("PARAGRAFO")
                sentences = groups.apply(lambda sdf: "".join(sdf.PAROLA))
                df_3 = pd.DataFrame(sentences)
                df_3.reset_index(inplace=True)
                df_3.drop('PARAGRAFO', axis=1, inplace=True)

            with open(path + corpus_title + '.txt', "w", encoding='utf-8') as file:
                # Testo
                if (len(opere_autore) != 0): # check che ci sia almeno un libro in SCRIPTA altrimenti da errore
                    for index, row in df_3.iterrows():
                        file.write(row[0])
                # Se e solo se ci sono libri aggiuntivi per quella fase letteraria procedo aggiungendoli
                if (label in meta_data_agg['AUTORE'].dropna().unique().tolist()):
                    # Dalle opere aggiuntive seleziono solo le rilevanti 
                    opere_autore = meta_data_agg[meta_data_agg['AUTORE'].str.contains(label, na=False, case = False)]
                    # Selezioni i nomi dei file unici
                    lista_path_opere_autore = opere_autore['Filename'].unique().tolist()
                    # Seleziono uno a uno i diversi file e li aggiungo
                    for filename in lista_path_opere_autore:
                        filedata = None  # azzero
                        with open(f"{path_libri_trattati}\{filename}", "r", encoding="utf8") as file_book:
                            # filedata = file_book.read()
                            for line in file_book:
                                file.write(line)        
        finally:
            pass

In [23]:
output_corpora_autori = "./data/Corpora/autori/"

build_corpus_autori(
    vocabolario, opere_autori_generi_testi, opere_aggiuntive, output_corpora_autori)

Dante Alighieri	6 opere totali
---
Francesco Petrarca	2 opere totali
---
Giovanni Boccaccio	14 opere totali
---
Ludovico Ariosto	27 opere totali
---
Torquato Tasso	10 opere totali
---
Giuseppe Parini	10 opere totali
---
Vittorio Alfieri	25 opere totali
---
Ugo Foscolo	4 opere totali
---
Alessandro Manzoni	44 opere totali
---
Giacomo Leopardi	12 opere totali
---
Italo Svevo	16 opere totali
---
Luigi Pirandello	37 opere totali
---
Gabriele D'Annunzio	30 opere totali
---
Pier Paolo Pasolini	2 opere totali
---
Italo Calvino	4 opere totali
---
Dino Buzzati	2 opere totali
---
Primo Levi	1 opere totali
---
Umberto Eco	2 opere totali
---


# Salvataggio file OPERE (SCRIPTA e aggiuntive) completi

In [92]:
opere_autori_generi_testi.to_csv('./data/CSV/OPERE_INTEGRATO.csv', sep='\t', encoding = 'utf-8', index = False)
opere_aggiuntive.to_csv('./data/CSV/OPERE_AGGIUNTIVE_INTEGRATO.csv', sep='\t', encoding = 'utf-8',  index = False)

# Lemmario

In [169]:
lemmario = pd.read_csv("./data/Lemmari/lemmario.csv", sep="\t", header=1, on_bad_lines='skip', quoting=3)
lemmario.tail(15)

,!,PON
39696,zurliniano,ADJ
39697,{,PON
39698,|,PON
39699,},PON
39700,}:-),SMI
39701,~~(=_=)~~,SMI
39702,«,PON
39703,»,PON
39704,àh,INT
39705,àncora,NOUN-F


In [172]:
parole_distinte = pd.read_csv("./data/Lemmari/parole_distinte.csv", header=None)
parole_distinte.head(15)

,0
0,A
1,AA
2,AAA
3,AAAA
4,AAAAA
5,AAAAAA
6,AAAAAAA
7,AAAAAAAA
8,AAAAAAAAA
9,AAAAAH
